In [ ]:
import calendar
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources
from scipy.stats import linregress
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tabulate import tabulate
import types

### Package requirements for reproducibility

In [ ]:
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
            
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

#### Define the initial dataset you'll be working on

In [ ]:
SatelliteJuly = pd.read_excel('Data.xlsx',sheet_name='July_sat')
SatelliteJune = pd.read_excel('Data.xlsx',sheet_name='June_sat')
SatelliteMay = pd.read_excel('Data.xlsx',sheet_name='May_sat')
SatelliteApril = pd.read_excel('Data.xlsx',sheet_name='April_satellite')
Bands = pd.read_csv('Sentinel_2A_band.csv',index_col=0,usecols=[0,5,6,7,8,9])
Seed = pd.read_excel('Data.xlsx',sheet_name='Chem_comp_wheat')
Dough = pd.read_excel('Data.xlsx',sheet_name='Dough',usecols=['W','P/L'])
Bread = pd.read_excel('Data.xlsx',sheet_name='Bread',usecols=[2,3,4,5,6,7,8,9,10])

In [ ]:
SatelliteData = pd.concat([SatelliteApril,SatelliteMay,SatelliteJune,SatelliteJuly],axis=1)

#### The figures get only normalised for the sake of convenience

In [ ]:
scaler = StandardScaler()
scaler.fit(SatelliteJuly)
scaledSatellite = pd.DataFrame(scaler.transform(SatelliteJuly))

In [ ]:
#### The PCA is eventually run

In [ ]:
pca = PCA(.99)
pca.fit(scaledSatellite)
PCA_Satellite = pd.DataFrame(pca.transform(scaledSatellite))
pca.n_components_
pca.explained_variance_ratio_.sum()

#### Let us now run the scatter plot of the PCA against the wheat composition

In [ ]:
colors = ['b','g','r']
for c1 in Seed:
    c0 = -1
    for c in PCA_Satellite:
        c0+=1
        plt.scatter(PCA_Satellite[c],Seed[c1],color=colors[c0],s=1,label=str(c1))
        plt.plot(np.unique(PCA_Satellite[c]),np.poly1d(np.polyfit(PCA_Satellite[c], Seed[c1], 1))
                 (np.unique(PCA_Satellite[c])),color=colors[c0],label='R2='+str(linregress(PCA_Satellite[c],Seed[c1])[2]**2))
    plt.legend()
    plt.show()

In [ ]:
colors = ['b','g','r']
for c1 in Dough:
    c0 = -1
    for c in PCA_Satellite:
        c0+=1
        plt.scatter(PCA_Satellite[c],Dough[c1],color=colors[c0],s=1,label=str(c1))
        plt.plot(np.unique(PCA_Satellite[c]),np.poly1d(np.polyfit(PCA_Satellite[c], Dough[c1], 1))
                 (np.unique(PCA_Satellite[c])),color=colors[c0],label='R2='+str(linregress(PCA_Satellite[c],Dough[c1])[2]**2))
    plt.legend()
    plt.show()

In [ ]:
colors = ['b','g','r']
for c1 in Bread:
    c0 = -1
    for c in PCA_Satellite:
        c0+=1
        plt.scatter(PCA_Satellite[c],Bread[c1],color=colors[c0],s=1,label=str(c1))
        plt.plot(np.unique(PCA_Satellite[c]),np.poly1d(np.polyfit(PCA_Satellite[c], Bread[c1], 1))
                 (np.unique(PCA_Satellite[c])),color=colors[c0],label='R2='+str(linregress(PCA_Satellite[c],Bread[c1])[2]**2))
    plt.legend()
    plt.show()

In [ ]:
for c1 in Seed:
    for c in SatelliteJuly:
        if linregress(SatelliteJuly[c],Seed[c1])[2]**2 > 0.1:
            plt.scatter(SatelliteJuly[c],Seed[c1],s=1,label=str(c)[:-7]+'_'+str(c1))
            plt.plot(np.unique(SatelliteJuly[c]),np.poly1d(np.polyfit(SatelliteJuly[c], Seed[c1], 1))
                     (np.unique(SatelliteJuly[c])),label='R2='+str(linregress(SatelliteJuly[c],Seed[c1])[2]**2))
            plt.legend()
            plt.show()

In [ ]:
for c1 in Dough:
    for c in SatelliteJuly:
        if linregress(SatelliteJuly[c],Dough[c1])[2]**2 > 0.1:
            plt.scatter(SatelliteJuly[c],Dough[c1],s=1,label=str(c)[:-7]+'_'+str(c1))
            plt.plot(np.unique(SatelliteJuly[c]),np.poly1d(np.polyfit(SatelliteJuly[c], Dough[c1], 1))
                     (np.unique(SatelliteJuly[c])),label='R2='+str(linregress(SatelliteJuly[c],Dough[c1])[2]**2))
            plt.legend()
            plt.show()

In [ ]:
for c1 in Bread:
    for c in SatelliteJuly:
        if linregress(SatelliteJuly[c],Bread[c1])[2]**2 > 0.1:
            plt.scatter(SatelliteJuly[c],Bread[c1],s=1,label=str(c)[:-7]+'_'+str(c1))
            plt.plot(np.unique(SatelliteJuly[c]),np.poly1d(np.polyfit(SatelliteJuly[c],Bread[c1], 1))
                 (np.unique(SatelliteJuly[c])),label='R2='+str(linregress(SatelliteJuly[c],Bread[c1])[2]**2))
            plt.legend()
            plt.show()

In [ ]:
# Let us analyse the data across all months

In [ ]:
c0 = 0
for c1 in Seed:
    for c in SatelliteData:
        if linregress(SatelliteData[c],Seed[c1])[2]**2 > 0.1:
            c0+=1
            print(c0)

In [ ]:
c0 = 0
for c1 in Dough:
    for c in SatelliteData:
        if linregress(SatelliteData[c],Dough[c1])[2]**2 > 0.1:
            c0+=1
            print(c0)

In [ ]:
c0 = 0
for c1 in Bread:
    for c in SatelliteData:
        if linregress(SatelliteData[c],Bread[c1])[2]**2 > 0.1:
            c0+=1
            print(c0)

### Let's see what month shows the best correlation between spectral data and seed data

In [ ]:
Satellite_list = [SatelliteApril,SatelliteMay,SatelliteJune,SatelliteJuly]
co1 = []
for sl in Satellite_list:
    co = []
    for c in sl:
        for c1 in Seed:
            if linregress(sl[c],Seed[c1])[2]**2 > 0.1:
                co.append(linregress(sl[c],Seed[c1])[2]**2)
    co1.append(co)
good_regressions = pd.DataFrame(co1,index=['April','May','June','July']).T

In [ ]:
Satellite_list = [SatelliteApril,SatelliteMay,SatelliteJune,SatelliteJuly]
co1 = []
for sl in Satellite_list:
    co = []
    for c in sl:
        for c1 in Seed:
            if linregress(sl[c],Seed[c1])[2]**2 > 0.1:
                co.append(linregress(sl[c],Seed[c1])[2]**2)
    co1.append(co)
good_regressions = pd.DataFrame(co1,index=['April','May','June','July']).T

In [ ]:
good_regressions

### Query to see what is the highest regression figure

In [ ]:
for sl in Satellite_list:
    for c in sl:
        for c1 in Seed:
            if linregress(sl[c],Seed[c1])[2]**2 > 0.5:
                print(c,c1)

### Regression with the Dough figures

In [ ]:
co1 = []
for sl in Satellite_list:
    co = []
    for c in sl:
        for c1 in Dough:
            if linregress(sl[c],Dough[c1])[2]**2 > 0.1:
                co.append(linregress(sl[c],Dough[c1])[2]**2)
    co1.append(co)
good_regressions_Dough = pd.DataFrame(co1,index=['April','May','June','July']).T
good_regressions_Dough

In [ ]:
for sl in Satellite_list:
    for c in sl:
        for c1 in Dough:
            if linregress(sl[c],Dough[c1])[2]**2 > 0.4:
                print(c,c1)

### Let examine how many meaningful relations one can find among bands and the features we are interested in

In [ ]:
names = ['Seed','Dough','Bread']
db = [Seed,Dough,Bread]
Fc = [(names[idb],c) for idb,d in enumerate(db) for c in d ]
Features = pd.concat([Seed,Dough,Bread],axis=1)
Features.columns = pd.MultiIndex.from_tuples(Fc,names=('Stage','Feature'))

In [ ]:
bl2 = []
bli = []
for ib in list(set(Bands.index)):
    bl1 = []
    for c1 in Bands.loc[ib]:
        bl = []
        bli.append((calendar.month_abbr[ib],c1))
        for c in Features:
            bl.append(linregress(Bands.loc[ib,c1],Features[c])[2]**2)
        bl1.append(bl)
    bl2.append(bl1)
Regression_df = pd.concat([pd.DataFrame(b1) for b1 in bl2])
Regression_df.index = pd.MultiIndex.from_tuples(bli,names=('Month','Band'))
Regression_df.columns = Features.columns

In [ ]:
Regression_df[Regression_df>0.1].count()

In [ ]:
Regressions_feat = pd.Series([Regression_df.loc[:,pd.IndexSlice[c0,:]][Regression_df.loc[:,pd.IndexSlice[c0,:]]>0.1].count().sum()
for c0 in list(set(Regression_df.columns.get_level_values(0)))], index = 
                             [c0 for c0 in list(set(Regression_df.columns.get_level_values(0)))])
Regressions_feat

In [ ]:
Regressions_month = pd.Series([Regression_df.loc[pd.IndexSlice[c2,:],:]
                              [Regression_df.loc[pd.IndexSlice[c2,:],:]>0.1].count().sum()
for c2 in list(set(Regression_df.index.get_level_values(0)))], index = 
                             [c2 for c2 in list(set(Regression_df.index.get_level_values(0)))])
Regressions_month

In [ ]:
Regressions_band = pd.Series([Regression_df.loc[pd.IndexSlice[:,c3],:]
                              [Regression_df.loc[pd.IndexSlice[:,c3],:]>0.1].count().sum()
for c3 in list(set(Regression_df.index.get_level_values(1)))], index = 
                             [c3 for c3 in list(set(Regression_df.index.get_level_values(1)))])
Regressions_band

In [155]:
Regressions_band_feat = pd.DataFrame([Regression_df.loc[pd.IndexSlice[:,c3],:]
                              [Regression_df.loc[pd.IndexSlice[:,c3],:]>0.1].count()
for c3 in list(set(Regression_df.index.get_level_values(1)))], index = 
                             [c3 for c3 in list(set(Regression_df.index.get_level_values(1)))])
Regressions_band_feat

Stage      Seed                                                       \
Feature p 1000s p hl  N  C Prot tot Prot sol acq Prot nacl Prot etoh   
B05           3    2  3  0        3            2         2         0   
B08           2    0  3  0        3            0         0         0   
B03           2    2  3  0        3            0         0         0   
B04           2    1  3  0        3            0         0         0   
B02           1    0  3  0        3            0         0         0   

Stage                       Dough     Bread                                     
Feature Prot ac ac Glut tot     W P/L um cr um mol hard Spr Coh Gum Chew d mol  
B05              0        2     2   2     0      0    0   0   0   0    0     0  
B08              0        0     0   2     1      0    0   0   1   0    1     0  
B03              0        0     2   0     0      0    0   1   1   0    0     1  
B04              0        0     1   1     0      0    0   0   1   0    0     0  
B02              0        0     1   1     0      0    0   0   0   0    0     0